In [1]:
import sys
CENTERNET_PATH = '/home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/src/lib'
sys.path.insert(0, CENTERNET_PATH)

import os
import cv2

from opts import opts
from detectors.detector_factory import detector_factory
import numpy as np

MODEL_PATH = '/home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/exp/ctdet/neu_hg_256/model_best.pth'
TASK = 'ctdet' # or 'multi_pose' for human pose estimation
opt = opts().init('{} --load_model {} --arch hourglass --gpus 0 --dataset neu_det'.format(TASK, MODEL_PATH).split(' '))
from datasets.dataset_factory import get_dataset
Dataset = get_dataset('neu_det', 'ctdet')
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
neu = Dataset(opt, 'val')
detector = detector_factory[opt.task](opt)
neu.images

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  /home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/src/lib/../../exp/ctdet/default
heads {'hm': 6, 'wh': 2, 'reg': 2}
heads {'hm': 6, 'wh': 2, 'reg': 2}
==> initializing pascal val data.
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loaded val 180 samples
Creating model...
loaded /home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/exp/ctdet/neu_hg_256/model_best.pth, epoch 80


[1,
 4,
 14,
 18,
 21,
 26,
 27,
 34,
 44,
 60,
 66,
 68,
 76,
 79,
 90,
 91,
 107,
 108,
 123,
 129,
 144,
 151,
 170,
 179,
 180,
 181,
 182,
 190,
 209,
 221,
 223,
 259,
 276,
 277,
 292,
 315,
 327,
 336,
 349,
 364,
 372,
 388,
 396,
 424,
 470,
 478,
 503,
 526,
 537,
 538,
 544,
 546,
 549,
 557,
 590,
 598,
 625,
 646,
 647,
 661,
 681,
 684,
 686,
 689,
 691,
 698,
 704,
 714,
 721,
 728,
 745,
 768,
 774,
 776,
 777,
 794,
 802,
 816,
 828,
 831,
 835,
 849,
 860,
 861,
 882,
 888,
 890,
 894,
 911,
 919,
 935,
 936,
 945,
 962,
 968,
 977,
 990,
 991,
 996,
 998,
 1014,
 1027,
 1045,
 1046,
 1051,
 1067,
 1069,
 1080,
 1090,
 1098,
 1099,
 1103,
 1115,
 1120,
 1136,
 1149,
 1151,
 1152,
 1157,
 1174,
 1175,
 1201,
 1222,
 1224,
 1248,
 1265,
 1287,
 1291,
 1299,
 1301,
 1308,
 1309,
 1322,
 1324,
 1348,
 1366,
 1367,
 1370,
 1403,
 1422,
 1438,
 1449,
 1476,
 1481,
 1483,
 1487,
 1500,
 1538,
 1540,
 1545,
 1566,
 1575,
 1576,
 1585,
 1595,
 1619,
 1630,
 1632,
 1637,
 1642

In [ ]:
#单张图片测试
file_name = neu.coco.loadImgs(ids=[1175])[0]['file_name']
img_path = os.path.join(neu.img_dir, file_name)
img = cv2.imread(img_path)
ret = detector.run(img)['results']
print(ret)

In [ ]:
boxes = ret[1][:,:-1].astype(np.int)
print(boxes)
scores = ret[1][:,-1]
print(scores)
N = 0
#print(ret)
for i in ret.keys():
    if ret[i] is None:
        continue
    scores = ret[i][:,-1]
    for j in range(ret[i].shape[0]):
         if scores[j] >= 0.3:
                 N += 1
print(N)

In [ ]:
def display_instances(image, ret, class_names,
                     title="", figsize=(16, 16), ax=None,
                     show_bbox=True, colors=None, captions=None):
    auto_show = False
    if not ax:
        _, ax = plt.subplot(1, figsize=figsize)
        auto_show = True
    #colors = colors or random_colors(N)
    #从每个类开始处理
    THRESHOLD = 0.4
    N = 0
    for i in ret.keys():
        if ret[i] is None:
            continue
        scores = ret[i][:,-1]
        for j in ret[i].shape[0]:
            if scores[j] >= THRESHOLD:
                N += 1
            
    for i in ret.keys():
        if ret[i] is None:
            continue
        class_name = class_names[i]
        boxes = ret[i][:,:-1].astype(np.int)
        scores = ret[i][:,-1]
        for i in ret[i].shape[0]: #一个一个判断
            if scores[i] >= THRESHOLD:  #显示阈值
                x1, y1, x2, y2 = boxes[i]
                color = random_colors()

In [ ]:
time_stats = ['tot', 'load', 'pre', 'net', 'dec', 'post', 'merge']
for img_id in neu.images:
    file_name = neu.coco.loadImgs(ids=[img_id])[0]['file_name']
    img_path = os.path.join(neu.img_dir, file_name)
    img = cv2.imread(img_path)
    ret = detector.run(img)
    time_str = ''
    for stat in time_stats:
        time_str = time_str + '{} {:.3f}s |'.format(stat, ret[stat])
    print(time_str)
    

In [ ]:
file_name = neu.coco.loadImgs(ids=[1])[0]['file_name']